In [42]:
# __pycache__ファイルを作成しないようにする。
import os
os.environ['PYTHONDONTWRITEBYTECODE'] = "1"

In [44]:
%cd /Users/naoki/git/Horse-Racing
import module as m
import module._class as c
import _dat
from environment.variables import *

/Users/naoki/git/Horse-Racing
/Users/naoki/git/Horse-Racing/Notebook


In [45]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re
import datetime
import lightgbm as lgb
import optuna.integration.lightgbm as lgb_o
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from bs4 import BeautifulSoup
from itertools import combinations
from itertools import permutations
from urllib.request import urlopen

In [47]:
# race_results(=r.data_c)の作成
r = c.Results(_dat.race_results)
# 前処理
r.preprocessing()
# 馬の過去成績の追加
hr = c.HorseResults(_dat.horse_results)
r.merge_horse_results(hr)
# 5世代分の血統データの追加
p = c.Peds(_dat.ped_results)
p.encode()
r.merge_peds(p.peds_e)
# カテゴリ変数の処理
r.process_categorical()

100%|██████████| 542/542 [00:40<00:00, 13.49it/s]


In [50]:
# r.data_cをtrain, test, validの3種のデータに分割
train, test = m.split_data(r.data_c)
train, valid = m.split_data(train)

In [51]:
# train, validを説明変数と目的変数に分割
X_train = train.drop(['rank', 'date', '単勝'], axis=1)
y_train = train['rank']
X_valid = valid.drop(['rank', 'date', '単勝'], axis=1)
y_valid = valid['rank']

In [ ]:
# lgbm用のデータに変形する
lgb_train = lgb_o.Dataset(X_train.values, y_train.values)
lgb_valid = lgb_o.Dataset(X_valid.values, y_valid.values)

In [ ]:
# パラメータ設定1
params = {
    'objective': 'binary',
    'random_state': 100,
    'verbose':0
}

In [ ]:
# パラメータ設定2
lgb_clf_o = lgb_o.train(params,
                        lgb_train,
                        valid_sets=(lgb_train, lgb_valid),
                        verbose_eval=100,
                        early_stopping_rounds=10
                        )

In [ ]:
lgb_clf_o.params

In [52]:
params={'objective': 'binary',
        'random_state': 100,
        'feature_pre_filter': False,
        'lambda_l1': 5.289135433159377,
        'lambda_l2': 4.708795060836079,
        'num_leaves': 16,
        'feature_fraction': 0.4,
        'bagging_fraction': 1.0,
        'bagging_freq': 0,
        'min_child_samples': 20}

In [53]:
# r.data_cを学習用データと検証用データに分割する
train, test = m.split_data(r.data_c)
X_train = train.drop(['rank', 'date', '単勝'], axis=1)
y_train = train['rank']
X_test = test.drop(['rank', 'date'], axis=1)
y_test = test['rank']

In [63]:
# lgbで実際に学習を行う
lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train.values, y_train.values)

LGBMClassifier(bagging_fraction=1.0, bagging_freq=0, feature_fraction=0.4,
               feature_pre_filter=False, lambda_l1=5.289135433159377,
               lambda_l2=4.708795060836079, num_leaves=16, objective='binary',
               random_state=100)

In [64]:
# ModelEvaluator
me = c.ModelEvaluator(lgb_clf, ['_dat/pickle/overall/return_tables.pickle'])
display(me.feature_importance(X_test.drop(['単勝'], axis=1), n_display=50))

,features,importance
5,jockey_id,126
7,n_horses,74
6,年齢,57
9,賞金_5R,51
27,賞金_race_type_5R,42
116,interval,42
3,course_len,35
4,horse_id,31
10,着差_5R,31
99,賞金_race_type_allR,30


In [65]:
g_proper = m.gain(me.tansho_return_proper, X_test)
g_tansho = m.gain(me.tansho_return, X_test)
g_fukusho = m.gain(me.fukusho_return, X_test)
g_umatan = m.gain(me.umatan_box, X_test)
g_umaren = m.gain(me.umaren_box, X_test)
g_wide = m.gain(me.wide_box, X_test)
g_sanrentan = m.gain(me.sanrentan_box, X_test)
g_sanrenpuku = m.gain(me.sanrenpuku_box, X_test)
g_umaren_nagashi = m.gain(me.umaren_nagashi, X_test)
g_umatan_nagashi = m.gain(me.umatan_nagashi, X_test)
g_wide_nagashi = m.gain(me.wide_nagashi, X_test)
g_sanrentan_nagashi = m.gain(me.sanrentan_nagashi, X_test)

 55%|█████▌    | 55/100 [06:17<03:33,  4.74s/it]

In [ ]:
plt.figure(figsize=(10, 8))
m.plot(g_proper, 'proper')
plt.hlines(1, xmin=0.5, xmax=3.5) # y=1の直線